## Aux 1 Bicyle accidents in Madrid in 2019 by district

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime

#### Download 2019 Madrid traffic accidents information from the Madrid's Council opendata website

In [2]:
# Download the information
!wget -q -O 'accidents2019.csv' https://datos.madrid.es/egob/catalogo/300228-19-accidentes-trafico-detalle.csv

In [3]:
# Check the encoding of the csv
!pip install chardet
import chardet 

with open("accidents2019.csv", 'rb') as file:
    print(chardet.detect(file.read()))

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


#### Create a DataFrame (df_accidents) from the CSV

In [4]:
# Read the csv downloaded into a df
df_accidents = pd.read_csv('accidents2019.csv', sep=';', encoding='ISO-8859-1')
df_accidents.head(5)

,Nº EXPEDIENTE,FECHA,HORA,CALLE,NÚMERO,DISTRITO,TIPO ACCIDENTE,ESTADO METEREOLÓGICO,TIPO VEHÍCULO,TIPO PERSONA,RANGO EDAD,SEXO,LESIVIDAD*,* La correspondencia de los códigos se encuentra descrito en la estructura del fichero.
0,2019S000020,01/01/2019,23:30,CALL. FUENCARRAL,149,CHAMBERÍ,Caída,Despejado,Ciclomotor,Conductor,DE 25 A 29 AÑOS,Hombre,1.0,NaN
1,2019S000017,01/01/2019,22:15,CALL. OCA / CALL. PINZON,-,CARABANCHEL,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 40 A 44 AÑOS,Mujer,14.0,NaN
2,2019S000017,01/01/2019,22:15,CALL. OCA / CALL. PINZON,-,CARABANCHEL,Colisión fronto-lateral,Despejado,Ciclomotor,Conductor,DE 35 A 39 AÑOS,Hombre,3.0,NaN
3,2019S001812,01/01/2019,21:40,CALL. BAILEN / CUSTA. SAN VICENTE,-,CENTRO,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 40 A 44 AÑOS,Hombre,14.0,NaN
4,2019S001812,01/01/2019,21:40,CALL. BAILEN / CUSTA. SAN VICENTE,-,CENTRO,Colisión fronto-lateral,Despejado,Turismo,Conductor,DE 30 A 34 AÑOS,Mujer,7.0,NaN


In [5]:
# Check the size of the df
df_accidents.shape

(51806, 14)

#### Data Cleansing

In [6]:
# Delete columns
df_accidents = df_accidents.drop(df_accidents.columns[[0, 2, 3, 4, 6, 7, 9, 10, 11, 12, 13]], axis=1)

# Rename columns
df_accidents.rename(columns={'Nº  EXPEDIENTE': 'EXP', 'FECHA':'DATE', 'DISTRITO':'DISTRICT', 'TIPO VEHÍCULO':'VEHICLE'}, inplace=True)

df_accidents.head(5)

,DATE,DISTRICT,VEHICLE
0,01/01/2019,CHAMBERÍ,Ciclomotor
1,01/01/2019,CARABANCHEL,Turismo
2,01/01/2019,CARABANCHEL,Ciclomotor
3,01/01/2019,CENTRO,Turismo
4,01/01/2019,CENTRO,Turismo


In [7]:
# Change type of columns: date to datetime
df_accidents['DATE'] = pd.to_datetime(df_accidents['DATE'])

df_accidents.dtypes

DATE        datetime64[ns]
DISTRICT            object
VEHICLE             object
dtype: object

#### Explore the number of accidentes by type of vehicle

In [8]:
df_accidents['VEHICLE'].value_counts()

Turismo                              36499
Motocicleta > 125cc                   3527
Furgoneta                             3125
Motocicleta hasta 125cc               2529
Autobús                               1408
Camión rígido                         1167
Bicicleta                              884
Ciclomotor                             809
Todo terreno                           689
Otros vehículos con motor              330
Tractocamión                           195
Maquinaria de obras                    117
Vehículo articulado                    106
Autobús articulado                      83
Sin especificar                         42
Ciclo                                   20
VMU eléctrico                           18
Cuadriciclo ligero                      15
Cuadriciclo no ligero                   14
Autocaravana                            13
Patinete                                 9
Bicicleta EPAC (pedaleo asistido)        7
Otros vehículos sin motor                7
Semiremolqu

#### Filter bicycle accidents information

In [9]:
# We only need information for accidents where the type of vehicle involved was a bicycle
df_accidents = df_accidents[df_accidents['VEHICLE']=='Bicicleta']

# We drop the column vehicle as it does not provide useful information now
df_accidents = df_accidents.drop(df_accidents.columns[[2]], axis=1)

# Reset index
df_accidents.reset_index(inplace=True, drop=True)
df_accidents.head(5)

,DATE,DISTRICT
0,2019-01-01,SALAMANCA
1,2019-02-01,HORTALEZA
2,2019-03-01,VILLA DE VALLECAS
3,2019-03-01,VILLA DE VALLECAS
4,2019-03-01,VILLA DE VALLECAS


In [10]:
# Check the new size of the df
df_accidents.shape

(884, 2)

In [11]:
# Check if there is any missing value
df_accidents.isna().sum()

DATE        0
DISTRICT    0
dtype: int64

#### Create a new DataFrame (df_accidents_summary) grouping bicycle accidents information by district

In [12]:
df_accidents_summary = df_accidents.groupby('DISTRICT').count().reset_index()
df_accidents_summary.rename(columns={'DATE':'NO ACCIDENTS'}, inplace=True)
df_accidents_summary.sort_values(by=['NO ACCIDENTS'], ascending=False, inplace=True)
df_accidents_summary.reset_index(inplace=True, drop=True)
df_accidents_summary

,DISTRICT,NO ACCIDENTS
0,CENTRO,142
1,ARGANZUELA,72
2,SALAMANCA,72
3,CHAMBERÍ,70
4,RETIRO,59
5,FUENCARRAL-EL PARDO,49
6,TETUÁN,45
7,MONCLOA-ARAVACA,45
8,CARABANCHEL,43
9,PUENTE DE VALLECAS,36


In [13]:
# Data cleansing: change the names with accent characters
df_accidents_summary.replace({'CHAMBERÍ':'CHAMBERI'}, inplace=True)
df_accidents_summary.replace({'TETUÁN':'TETUAN'}, inplace=True)
df_accidents_summary.replace({'CHAMARTÍN':'CHAMARTIN'}, inplace=True)
df_accidents_summary.replace({'VICÁLVARO':'VICALVARO'}, inplace=True)
df_accidents_summary

,DISTRICT,NO ACCIDENTS
0,CENTRO,142
1,ARGANZUELA,72
2,SALAMANCA,72
3,CHAMBERI,70
4,RETIRO,59
5,FUENCARRAL-EL PARDO,49
6,TETUAN,45
7,MONCLOA-ARAVACA,45
8,CARABANCHEL,43
9,PUENTE DE VALLECAS,36


#### We save the DataFrame into a CSV file (Bicycle_Accidents.csv). Cell is hidden as it contains credentials.

In [14]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'Bicycle_Accidents.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'capstoneprojectnotebook-donotdelete-pr-o8lsietovhyq9h',
 'asset_id': '6fbf1279-b301-41ea-8a52-10f143cea047'}